In [86]:
import pandas as pd
import numpy as np

from itertools import combinations_with_replacement
from heapq import heapify, heappop, heappush

After doing some visualizations in an [isometric 3d cube drawer][1], first, I decided to do some programming to add layers in a brute-force way (not even optimal brute force). To do this, I just find the location of every current cube of a given shape, and move it 6 possible ways (up, down, left, right, forward, back). That gives us the next layer.

This very basic way was not intended to solve the problem but to hopefully get a pattern in the numbers--recursive geometry problems in general have a lot of patterns (call it the beauty of the universe). By testing out a few blocks we see the following pattern:
$$
\begin{array} {| c | c | c | c |}
\hline
\text{layer} & 3 \times 2 \times 1 & 5 \times 1 \times 1 & 1 \times 1 \times 1 \\
\hline
1 & 22 & 22 & 6 \\
2 & 46 & 50 & 18 \\
3 & 78 & 86 & 38 \\
4 & 118 & 130 & 66 \\
\vdots & \vdots & \vdots & \vdots \\
\hline
\end{array}
$$

The first pattern thing I did was try to subtract each row from each other. A pattern emerges!
$$
\begin{array} {| c | c | c | c |}
\hline
\text{layer} & 3 \times 2 \times 1 & 5 \times 1 \times 1 & 1 \times 1 \times 1 \\
\hline
1 \to 2 & 24 & 28 & 12 \\
2 \to 3 & 32 & 36 & 20 \\
3 \to 4 & 40 & 44 & 24 \\
\vdots & \vdots & \vdots & \vdots \\
\hline
\end{array}
$$

If we then take second differences, we see
$$
\begin{array} {| c | c | c | c |}
\hline
\text{layer} & 3 \times 2 \times 1 & 5 \times 1 \times 1 & 1 \times 1 \times 1 \\
\hline
(1 \to 2) \to (2 \to 3)  & 8 & 8 & 8 \\
(2 \to 3) \to (3 \to 4) & 8 & 8 & 8 \\
\vdots & \vdots & \vdots & \vdots \\
\hline
\end{array}
$$

(although I only display a few rows here and starting cuboids, I did check many more to see if the pattern holds)

This is a great pattern, because it means that once we get the change from layers $1 \to 2$, we can just add $8$ to it each time to get the following differences in layers. We could use brute-force to get the first two layers, but there happens to be an even better formula if we inspect the second table. It turns out that the change from layers $1 \to 2$ is $4(l + w + h)$. By "unrolling" the second differences to get back to the numbers, this gives a nice recurrence relation:
$$
\begin{align*}
F(1) &= 2(lw + lh + wh) \\
F(n) &= F(n-1) + 4(l + w + h) + 8(n-2)
\end{align*}
$$

From there calculating the volume of each cuboid up to $n$ is easy enough, you just keep counting up. I use an upper limit in my code which increases itself if no $n$ is found that has $C(n) = 1000$. So it stars at `max_vol = 100` but increments itself up until we find at least one hit for $C(n) = 1000$. Then take the minimum of those hits. It works well here.


[1]: https://www.nctm.org/Classroom-Resources/Illuminations/Interactives/Isometric-Drawing-Tool/

In [2]:
# physically add a layer to the shape
def basic_add_layer(cuboid):
    new_cuboid = cuboid.copy()
    ls, ws, hs = np.where(cuboid == 1)
    current_cubes = [(ls[i], ws[i], hs[i]) for i in range(len(ls))]

    # six possible movements
    movements = [(1,0,0), (-1,0,0), (0,1,0), (0,-1,0), (0,0,1), (0,0,-1)]

    # count blocks changed to 1
    blocks_added = set()

    # go through every current block
    for sl, sw, sh in current_cubes:
        for dl,dw,dh in movements:
            nl, nw, nh = sl+dl, sw+dw, sh+dh
            if cuboid[nl, nw, nh] == 0:
                blocks_added.add((nl, nw, nh))

            new_cuboid[nl, nw, nh] = 1
    
    return new_cuboid.copy(), len(blocks_added)

In [3]:
l,w,h = 3,2,1

cuboid = np.zeros((300,300,300), int)
cuboid[150-l:150, 150-w:150, 150-h:150] = 1

n_cubes = 0
n_cubes2 = 2*(l*w + l*h + w*h)
cntr = -1

nc = cuboid.copy()

# pretty slow even for just one cuboid and 10 layers -- not scalable
# also requires creation of an array and scaling it up which is not scalable
while cntr < 10:
    nc, n_cubes = basic_add_layer(nc)

    print(n_cubes, n_cubes2)
    
    cntr += 1
    n_cubes2 += 4*(l+w+h) + 8*cntr


22 22
46 46
78 78
118 118
166 166
222 222
286 286
358 358
438 438
526 526
622 622


In [157]:
# recursive approach
layer_cache = {}
def better_add_layer(dims, layer):
    if (dims, layer) not in layer_cache:
        l,w,h = dims
        # by default add on one layer
        if layer <= 1:
            layer_cache[(dims, layer)] =  2*(l*w + l*h + w*h)
        else:
            layer_cache[(dims, layer)] = better_add_layer(dims, layer-1) + 4*(l+w+h) + 8*(layer - 2)
    
    return layer_cache[(dims, layer)]

In [160]:
prev_max_vol = 0
max_vol = 100
step_size = 10000

while True:
    c = {}
    l,w,h = 1,1,1
    print(max_vol)

    while better_add_layer((l,1,1),1) <= max_vol:
        while better_add_layer((l,w,1),1) <= max_vol and w <= l:
            while better_add_layer((l,w,h),1) <= max_vol and h <= w:
                layer = 1
                while True:
                    vol =  better_add_layer((l,w,h),layer)
                    if vol > max_vol:
                        break

                    c[vol] = c.get(vol, 0) + 1
                    layer += 1

                h += 1
        
            w += 1
            h = 1

        l += 1
        w = 1
        h = 1


    if 10 in c.values():
        print(10, 'found')
    if 100 in c.values():
        print(100, 'found')

    if 1000 in c.values():
        break

    max_vol += step_size

for k in sorted(c.keys()):
    if c[k] == 1000:
        print(k)
        break


100
10100
10 found
100 found
20100
10 found
100 found
18522


In [161]:
for k in sorted(c.keys()):
    if c[k] == 1000:
        print(k)
        break

18522
